# Reranking using SciBERT

First, follow [this Notebook](https://github.com/castorini/anserini-notebooks/blob/master/Pyserini%2BSciBERT_on_COVID_19_Demo.ipynb) to reproduce SciBERT, then try `BertForNextSentence` to see if it is better.



**Prerequisite**

For evaluation, have [trec_eval](https://github.com/usnistgov/trec_eval) installed. Alternatively, use [Python](https://github.com/cvangysel/pytrec_eval) version.

Install original trec_eval:
1. Go to parent directory of this repo
2. git clone https://github.com/usnistgov/trec_eval.git
3. make


In [1]:
# Get Round 5 Anserini Baseline from https://github.com/castorini/anserini/blob/master/docs/experiments-covid.md
!wget -P ../tmp/ https://www.dropbox.com/s/lbgevu4wiztd9e4/anserini.covid-r5.abstract.qq.bm25.txt
    
# Get Round 5 topics
!wget -P ../tmp/ https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml

--2020-08-12 05:58:11--  https://www.dropbox.com/s/lbgevu4wiztd9e4/anserini.covid-r5.abstract.qq.bm25.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.1, 2620:100:601c:1::a27d:601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/lbgevu4wiztd9e4/anserini.covid-r5.abstract.qq.bm25.txt [following]
--2020-08-12 05:58:11--  https://www.dropbox.com/s/raw/lbgevu4wiztd9e4/anserini.covid-r5.abstract.qq.bm25.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca7e2e8cd3d954a714ee26ffb8e.dl.dropboxusercontent.com/cd/0/inline/A9RKJs5QEinIB2zg95aQOmGPWoeW2bJj65wcFCXjP5T6P5vG11d5ZeNBQ7piV7ExiOgtFHYAtqmkFqjnlSIIulf-A_IgZbGttv5LIuDruwqeF3AO9HXSgZmELb0uPLD7d7g/file# [following]
--2020-08-12 05:58:11--  https://uca7e2e8cd3d954a714ee26ffb8e.dl.dropboxusercontent.com/cd/0/inline/A9RKJs5QEinIB2zg95aQOmGPWoeW2bJj65wc

In [1]:
import csv 
import torch
from xml.etree import ElementTree
import pickle
import numpy
from tqdm import tqdm
from transformers import *
from collections import defaultdict
import warnings
import os
warnings.simplefilter(action='ignore', category=FutureWarning)

PATH_TO_CORD_19_DATA = "../../CORD-19/2020-07-16/"
PATH_TO_ANSERINI_RUN = "../tmp/anserini.covid-r5.abstract.qq.bm25.txt"
PATH_TO_TOPICS = "../tmp/topics-rnd5.xml"
PATH_TO_TREC = "../../trec_eval/trec_eval"
TOP_K = 200 # How many top K to rerank

In [2]:
# Take a look at Anserini baseline. We will see if SciBERT can imporve

ret = os.system(PATH_TO_TREC + " -c -m all_trec ../tmp/qrels-covid_d4_j0.5-4.txt  ../tmp/anserini.covid-r5.abstract.qq.bm25.txt > ../tmp/out.txt")

if ret == 0:
    with open("../tmp/out.txt", "r") as f:
        print(f.read())
else:
    print("ERROR", ret)

runid                 	all	anserini.covid-r5.abstract.qq.bm25.txt
num_q                 	all	45
num_ret               	all	443518
num_rel               	all	15765
num_rel_ret           	all	11543
map                   	all	0.1903
gm_map                	all	0.1470
Rprec                 	all	0.2704
bpref                 	all	0.5195
recip_rank            	all	0.6570
iprec_at_recall_0.00  	all	0.7802
iprec_at_recall_0.10  	all	0.4139
iprec_at_recall_0.20  	all	0.3445
iprec_at_recall_0.30  	all	0.2740
iprec_at_recall_0.40  	all	0.2093
iprec_at_recall_0.50  	all	0.1592
iprec_at_recall_0.60  	all	0.1172
iprec_at_recall_0.70  	all	0.0768
iprec_at_recall_0.80  	all	0.0367
iprec_at_recall_0.90  	all	0.0143
iprec_at_recall_1.00  	all	0.0011
P_5                   	all	0.5067
P_10                  	all	0.4933
P_15                  	all	0.4874
P_20                  	all	0.4822
P_30                  	all	0.4593
P_100                 	all	0.3787
P_200                 	all	0.3156
P_500                 

In [4]:
# Read Anserini Baseline

with open(PATH_TO_ANSERINI_RUN, "r") as f:
    anserini_run = f.readlines()
    
set_of_top_k_uid_all_topics = set()


for line in anserini_run:
    qid, _, uid, rank, _, _ = line.strip().split()
    if int(rank) > TOP_K:
        continue
    
    set_of_top_k_uid_all_topics.add(uid)


In [5]:
# Read topics

def read_topics(path_to_topic):
    tree = ElementTree.parse(path_to_topic)
    topics = list()
    for topic in tree.getroot():
        d = dict()
        d["number"] = topic.attrib["number"]
        for field in topic:
            d[field.tag] = field.text
        topics.append(d)

    return topics

topics = read_topics(PATH_TO_TOPICS)

In [6]:
# Read Abstract

def get_abstract(path):
    abstracts = dict()
    with open(path + "metadata.csv") as f_in:
        reader = csv.DictReader(f_in)
        for row in reader:
            cord_uid = row['cord_uid']
            if cord_uid in set_of_top_k_uid_all_topics:
                continue

            abstract = row['abstract']
            abstracts[cord_uid] = abstract
            
    return abstracts


abstracts_dict = get_abstract(PATH_TO_CORD_19_DATA)


# Reproduce Anserini's SciBERT  [Notebook](https://github.com/castorini/anserini-notebooks/blob/master/Pyserini%2BSciBERT_on_COVID_19_Demo.ipynb)

What we want here is - given Anserini Run on abstract, use SciBERT to rerank first K abstracts, and evaluate using TREC metric as usual. This [Notebook](https://github.com/castorini/anserini-notebooks/blob/master/Pyserini%2BSciBERT_on_COVID_19_Demo.ipynb) just demonstrate how to use SciBERT. It's not ready to give us results.  A little bit more work has to be done.


In [7]:
tokenizer = AutoTokenizer.from_pretrained('monologg/biobert_v1.1_pubmed', do_lower_case=False)
model = AutoModel.from_pretrained('monologg/biobert_v1.1_pubmed')

In [8]:
CHUNK_SIZE = 510

# Waring can be ignored. We won't pass the entire sequence to the model anyway.

def extract_scibert(text, tokenizer, model):
    text_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    text_words = tokenizer.convert_ids_to_tokens(text_ids[0])[1:-1]

    n_chunks = int(numpy.ceil(float(text_ids.size(1))/CHUNK_SIZE))
    states = []
    
    for ci in range(n_chunks):
        text_ids_ = text_ids[0, 1+ci*CHUNK_SIZE:1+(ci+1)*CHUNK_SIZE]            
        text_ids_ = torch.cat([text_ids[0, 0].unsqueeze(0), text_ids_])
        if text_ids[0, -1] != text_ids[0, -1]:
            text_ids_ = torch.cat([text_ids_, text_ids[0,-1].unsqueeze(0)])
        
        with torch.no_grad():
            state = model(text_ids_.unsqueeze(0))[0]
            state = state[:, 1:-1, :]
        states.append(state)

    state = torch.cat(states, axis=1)
    return text_ids, text_words, state[0]

In [ ]:
encoded_abstract = dict()

for k, text in abstracts_dict.items():
    encoded_abstract[text] = extract_scibert(text, tokenizer, model)

Token indices sequence length is longer than the specified maximum sequence length for this model (624 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (602 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (621 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (545 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (610 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (530 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (540 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (579 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (536 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (529 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (521 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (543 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (512 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (552 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (699 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (520 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (561 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (646 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (791 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (518 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (558 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (626 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (661 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

...

Token indices sequence length is longer than the specified maximum sequence length for this model (579 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (590 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (677 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (545 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (624 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (535 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (632 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (511 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (892 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (595 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (609 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (553 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (691 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (771 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (669 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (784 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (560 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (704 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (644 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1133 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

Token indices sequence length is longer than the specified maximum sequence length for this model (578 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (532 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (542 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (536 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [ ]:

encoded_queries = dict()
encoded_naratives = dict()

for topic in topics:
    encoded_queries[topic["query"]] = extract_scibert(topic["query"], tokenizer, model)
    encoded_naratives[topic["narrative"]] = extract_scibert(topic["narrative"], tokenizer, model)

In [ ]:
# save
with open("../tmp/scibert.pkl", "wb") as f:
    pickle.dump({
        "encoded_abstract": encoded_abstract, 
        "encoded_queries": encoded_queries, 
        "encoded_naratives": encoded_naratives}, f)


In [ ]:
# load
with open("../tmp/scibert.pkl", "rb") as f:
    d = pickle.load(f)
    encoded_abstract = d["encoded_abstract"]
    encoded_queries = d["encoded_queries"]
    encoded_naratives = d["encoded_naratives"]

In [ ]:
def cross_match(state1, state2):
    state1 = state1 / torch.sqrt((state1 ** 2).sum(1, keepdims=True))
    state2 = state2 / torch.sqrt((state2 ** 2).sum(1, keepdims=True))
    sim = (state1.unsqueeze(1) * state2.unsqueeze(0)).sum(-1)
    return sim

In [ ]:
# reranking on query
# assume anserini_run contains >= 1000 hits each topic

rerank = defaultdict(list)  # first 200 hits
keeprank = defaultdict(list) # 200 - 1000 hits
query_dict = {e["number"]: e["query"] for e in topics}
narative_dict = {e["number"]: e["narrative"] for e in topics}

template = "{} Q0 {} {} {} anserini_scibert"
run = list()

for line in anserini_run:
    qid, _, uid, j, score, _ = line.strip().split()

    if int(j) < TOP_K:
        if not abstracts_dict[uid]:
            continue # Some uid don't have abstract. Why it shows up in Anserini run?
            
        _, _, enc_abs = encoded_abstract[abstracts_dict[uid]]
        _, _, enc_query = encoded_queries[query_dict[qid]]
        sim = cross_match(enc_query, enc_abs)

        rel_score = torch.max(sim).item()
        rerank[qid].append([uid, rel_score])

    elif 1100 >= int(j) >= TOP_K:  # just above 1000, becaus we have continue clause above
        keeprank[qid].append([uid, score, j])

    
    
for qid in rerank:
    rank = 1
    for uid, score in sorted(rerank[qid], key=lambda x:-x[1]):
        run.append(template.format(qid, uid, rank, score + 10))
        rank += 1
        
    for uid, score, j in keeprank[qid]:
        run.append(template.format(qid, uid, rank, score))
        rank += 1
        if rank == 1000:
            break


In [ ]:
with open("../tmp/scibert_sim_rerank.txt", "w") as f:
    f.write("\n".join(run))

In [ ]:

ret = os.system(PATH_TO_TREC + " -c -m all_trec ../tmp/qrels-covid_d4_j0.5-4.txt  ../tmp/scibert_sim_rerank.txt > ../tmp/out.txt")
if ret == 0:
    with open("../tmp/out.txt", "r") as f:
        print(f.read())
else:
    print("ERROR")

# Experiment with BertForNextSentence

In [50]:
!ls ../tmp

anserini.covid-r4.abstract.qdel.bm25.txt
baseline.txt
out.txt
qrels-covid_d4_j0.5-4.txt
qrels-covid_d4_j3.5-4.txt
report_run_0.txt
run0
run1
scibert.pkl
scibert_sim_rerank.txt
topics-rnd4.xml
wv_300_15_cleaned_uncased_trained_2020-06-19.model
wv_300_15_cleaned_uncased_trained_2020-06-19.model.vectors.npy
